# Text tokenization and vectorization
We use Spacy to tokenize documents in batch, exploiting the pipe method. We then save the tokens to disk to avoid recomputing, lastly, we use a tfidf vectorizer to compute document vectors, and, as before, we save the results to disk.

In [1]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from src.dataset import Dataset
from src.tokenizers import BatchTokenizer
from src.vectorizers import TokenTfidfVectorizer

### Loading the dataset
The dataset is loaded in a list of documents, we do not need all the information about them right now.

In [2]:
# number of documents to load, -1 means all of them
n_documents = 1000

# processed_dataset = "processed" #10seconds to load, contains approx. 19k documents
processed_dataset = "test_processed" # 1second to load, contains approx 1k documents

dataset = Dataset(dataset_path="../data/raw/data.jsonl", save_path=f"../data/processed/{processed_dataset}.json")
texts_list = dataset.load_text_list(size=n_documents)

In [4]:
len(texts_list)

1000

## Tokenize in batch
The tokenizer processes the documents in a parallel fashion, 
the result is a token list for each document, we save those to disk and later load them. 

In [3]:
bt = BatchTokenizer()
tokens = bt.tokenize(texts_list)
bt.save_tokens(tokens)
# tokens = BatchTokenizer.load_tokens()

## TfIdf vectorization

We now use TfIdf vectorizer on the obtained tokens.

In [4]:
dv = TokenTfidfVectorizer(tokens)

vectors = dv.vectors()
dv.save_vectors_vectorizer(vectors)

print(f"Vocabulary length: {len(dv.vectorizer.vocabulary_)}")


Vocabulary length: 1507


In [5]:
loaded_vectors, loaded_vec = TokenTfidfVectorizer.load_vectors_vectorizer()
X = pd.DataFrame(loaded_vectors.toarray(), columns=loaded_vec.get_feature_names())
X.head()

,104th,143,2d,302(a,302(a)(1,3d,603(a,603(h,609(a,652(a)(1,...,writing,x,x.,year,years,young,youth,zant,§,éxclude
0,0.00395,0.000000,0.067452,0.0079,0.01975,0.01185,0.00395,0.00395,0.000000,0.00395,...,0.000000,0.0237,0.00395,0.036536,0.01185,0.000000,0.000000,0.000000,0.01185,0.000000
1,0.00000,0.004738,0.080903,0.0000,0.00000,0.00000,0.00000,0.00000,0.004738,0.00000,...,0.004738,0.0000,0.00000,0.016855,0.00000,0.009476,0.004738,0.004738,0.00000,0.004738
